# Demo notebook: Facial landmarking

This demo introduces the cascaded regression models for facial landmark detection. 
* Both stationary as well as live webcam inputs will be used
* Both dlib's HOG based frontal face detector and it's 5-point/68-point shape predictors will be used.

## Datasets used by dlib's shape predictors
* 5-point shape predictor (`shape_predictor_5_face_landmarks.dat.bz2`):
    * dlib 5-point face landmark dataset
    * Consists of 7198 faces
* 68-point shape predictor (`shape_predictor_68_face_landmarks.dat.bz2`):
    * i-bug 300-W dataset (with landmark annotations) 
    * Link: https://ibug.doc.ic.ac.uk/resources/facial-point-annotations/ (however not freely accessible)

#### You can find the dlib predictors here:
http://dlib.net/files/

## Configuration (Ulf)

In [ ]:
# General configuration
import os

# data_directory: str
#     Path to a directory to store data.
data_directory = '.'

# install_missing_packages: bool
#     A flag indicating if missing packages should be automatically installed
install_missing_packages = True

# use_conda: bool
#     A flag indicating if conda should be used for software installation.
#     If False, pip will be used. The default is to use conda if jupyter
#     is run in a conda environment.
use_conda = 'CONDA_EXE' in os.environ

## Preparations (Ulf)

An auxiliary function to check for an install packages if missing:

In [ ]:
import importlib

def check_package(package, pip_pkg: str = None, conda_pkg: str = None):
    """Check if a given package is installed. If missing install
    it (if global flag `install_missing_packages` is True) either with
    pip or with conda (depending on `use_conda`).
    """
    if importlib.util.find_spec(package) is not None:
        return  # ok, package is already installed

    if not install_missing_packages:
        raise RuntimeError(f"{package} is not installed!")

    if use_conda:
        import conda.cli
        conda.cli.main('conda', 'install',  '-y', conda_pkg or package)
    else:
        import subprocess
        import sys            
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pip_pkg or package])
        
# This is to exit cells without error tracebacks (cosmetic purpose)
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Requirements

To run all of this notebook, you need the following libraries to be installed:
* ImageIO (`imageio` and `imageio-ffmpeg`): for reading images and accessing the webcam
* Scikit-image (`scikit-image`) for some image manipulation
* MatPlotLib (`matplotlib`): mainly for displaying images in the notebook
* Dlib (`dlib`) providing the HOG face detector
* OpenCV (`opencv`) for real time applications
* Imutils (`imutils`) for image manipulation with OpenCV

In [ ]:
%%writefile cascade_demo.yml
name: cascade_demo
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.6
  - jupyter
  - imageio
  - imageio-ffmpeg
  - matplotlib
  - scikit-image
  - dlib
  - opencv
  - imutils

In [ ]:
# import the necessary packages
from imutils import face_utils
import dlib
import cv2
import imageio
import matplotlib.pyplot as plt

# initialize dlib's HOG-based face detector 
hog_detector = dlib.get_frontal_face_detector()

## Face Detection Function 
I use Ulfs Detection function since it increases the performance by rescaling the image. Then, I extended it slightly by returning the detected face rectangles - later, the facial landmark predictor uses these rectangles

In [ ]:
import numpy as np
from skimage import draw, transform, color
import time

def process(image, detector, upsample_num: int = 1, scale = 1.):
    """Process an image by applying the given detector and
    visually marking detection in that image.

    Arguments
    ---------
    image: np.ndarray
        An array to be interpreted as RGB image (not BGR - the dlib engine
        seems to follow the standard color model!).
    
    detector:
        A dlib detector object.
    
    upsample_num: int
        Number of upsamples during detection. Can be any non-negative
        integer (including 0).
        
    scale: float
        A factor by which the image is scaled prior to detection.
        
    Results
    -------
    image: np.ndarray
        The image with detected faces marked
        
    detections: int
        Number of faces detected
        
    duration: float
        Time (in seconds) needed for detection
    
    rects: the detected rectangled faces 
    """
    # Resize image for faster face detection
    scaled_image = transform.rescale(image, scale=(scale, scale, 1),
                                     preserve_range=True).astype(np.uint8)

    
    # detect faces in the grayscale image
    start = time.time()
    rects = detector(scaled_image, upsample_num)
    end = time.time()

    # decorate the image: loop over the face detections
    for rect in rects:
        # rect is of type <class 'dlib.rectangle'>
        #
        # compute the bounding box of the face and draw it on the
        # original image
        top = max(0, int(rect.top() / scale))
        right = min(image.shape[1], int(rect.right() / scale))
        bottom = min(image.shape[0], int(rect.bottom() / scale))
        left = max(0, int(rect.left() / scale))
        image[(top,top+1, bottom-2, bottom-1), left:right] = (0, 255, 0)
        image[top:bottom, (left, left+1, right-2, right-1)] = (0, 255, 0)

    return image, rects, len(rects), (end-start)

## Detecting the faces using Ulfs function

In [ ]:
image = imageio.imread('images/LargestSelfie.jpg')   

upsample_num = 2
scale = 1.0

result_img, rects, face_detections, duration = \
process(image.copy(), hog_detector, upsample_num=upsample_num, scale=scale)

plt.figure(figsize=(15,15))
plt.title(f"Largest Selfie with Boxes\n"
          f"{face_detections} detection{'' if face_detections==1 else 's'}"
          f" in {duration*1000:.2f} ms")
plt.imshow(result_img)
plt.show()

## 5 key point landmark predictor (stationary image):
* Using the dlib 5 point shape predictor
* and the face detection function introduced above <br>
The predictor gets the returning face rectangles to analyse and draws 5 circles on the corresponding detection locations indicating the facial key landmarks  

In [ ]:
# create the facial landmark predictor
predictor_5 = "shape_predictor_5_face_landmarks.dat"

predictor = dlib.shape_predictor(predictor_5)

# get the input image
image = imageio.imread('images/LargestSelfie.jpg')   

# Convert it into gray scale (required for the predictor)
gray_Input = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# use the dlib detector to detect faces in the image box
result_img, rects, face_detections, duration = \
process(image.copy(), hog_detector, upsample_num=upsample_num, scale=scale)

# loop over those detected face candidates
for (i, detected) in enumerate(rects):
    
    # find the landmarks within each rectangle by using the dlib predictor
    shape = predictor(gray_Input, detected)
    
    # convert the resulting landmark coordinates (x,y) to a NumPy array
    shape = face_utils.shape_to_np(shape)

    # loop over all landmark coordinates (x,y) and mark them on the image
    # using cv2.circle(input_image, center coordinats, circle radius, circle color, circle thickness)
    radius = 2
    color = (193, 249, 162) # something like mint
    thickness = 2
    
    for (x, y) in shape:
        cv2.circle(image, (x, y), radius, color, thickness)


        
# Plotting 
plt.figure(figsize=(15,15))
plt.title(f"Largest Selfie with landmark predictions\n"
          f"Used 5 point detector\n")
plt.imshow(result_img)
plt.imshow(image)

## 68 key point landmark predictor (stationary image):
* Using the dlib 68 point shape predictor 
* and the face detection function introduced above <br>
The predictor gets the returning face rectangles to analyse and draws 68 circles on the corresponding detection locations indicating the facial key landmarks  

In [ ]:
# create the facial landmark predictor
predictor_68 = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_68)

# get the input image
# https://4.bp.blogspot.com/-aoMqfIsY5Ls/VHIBk6BQ3dI/AAAAAAAAf3Q/cRV668YRm4w/s1600/10700088_1029155283762368_4785788257713012496_o.jpg
image = imageio.imread('images/LargestSelfie.jpg')   

# Convert it into gray scale (required for the predictor)
gray_Input = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# use the dlib detector to detect faces in the image box
result_image, rects, detections, duration = \
process(image.copy(), hog_detector, upsample_num=upsample_num, scale=scale)

# loop over those detected face candidates
for (i, detected) in enumerate(rects):
    # find the landmarks within each rectangle by using the dlib predictor
    shape = predictor(gray_Input, detected)
    # convert the resulting landmark coordinates (x,y) to a NumPy array
    shape = face_utils.shape_to_np(shape)

    # loop over all landmark coordinates (x,y) and mark them on the image
    # using cv2.circle(input_image, center coordinats, circle radius, circle color, circle thickness)
    radius = 2
    color = (193, 249, 162) # something like mint
    thickness = 2
    
    for (x, y) in shape:
        cv2.circle(image, (x, y), radius, color, thickness)

        
        
# Plotting
plt.figure(figsize=(15,15))
plt.title(f"Largest Selfie with landmark predictions\n"
          f"Used 68 point detector\n")
plt.imshow(result_img)
plt.imshow(image)

## Facial Landmark Detection with live camera images:
* Here you can decide whether you want to use the 5 oder the 68 point predictor.
* Uses the same structure as the stationary examples
* However, using the video capture, aka webcam, as input and predicts the landmark locations while the camera is active

In [ ]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
predictor_5 = "shape_predictor_5_face_landmarks.dat"
predictor_68 = "shape_predictor_68_face_landmarks.dat"

# Using the dlib face detector
detector = dlib.get_frontal_face_detector()

# Choose between 5 and 68 point predictor here! 
predictor = 68

if predictor == 5:
    predictor = dlib.shape_predictor(predictor_5)
elif predictor == 68:
    predictor = dlib.shape_predictor(predictor_68)
else:
    print('Choose the right point-predictor!')
    raise StopExecution
        

cap = cv2.VideoCapture(0)

while True:
    # get the camera input
    _, image = cap.read()
    # Convert it into gray scale (required for the predictor)
    gray_Input = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    # use the dlib detector to detect faces in the image box
    faces = detector(gray_Input, 0)
    
    # loop over those detected face candidates
    for (i, detected) in enumerate(faces):
        # find the landmarks within each rectangle by using the dlib predictor
        shape = predictor(gray_Input, detected)
        # convert the resulting landmark coordinates (x,y) to a NumPy array
        shape = face_utils.shape_to_np(shape)
    
        # loop over all landmark coordinates (x,y) and mark them on the image
        # using cv2.circle(input_image, center coordinats, circle radius, circle color, circle thickness)
        radius = 2
        color = (193, 249, 162) # something like mint
        thickness = 2
    
        for (x, y) in shape:
            cv2.circle(image, (x, y), radius, color, thickness)
    
    # Update and display the output image with all detected facial landmarks 
    # on the respective face candidate (multiple are possible)
    cv2.imshow("Window", image)
    if cv2.waitKey(1) & 0xFF == ord('q'): # Press q to deactivate camera and close window
        break
        
# Deactivate the camera
cap.release()
# Close all windows
cv2.destroyAllWindows()

# Necessary for mac users to overcome the window closing barrier
for i in range (1,5):
    cv2.waitKey(1)

## $\uparrow$ Press q to stop! $\uparrow$
